<h2> Imports and loading event-log function </h2>

In [58]:
import numpy as np
import pandas as pd
import pm4py
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import statistics
from collections import Counter

def import_xes(file_path):
    log = pm4py.read_xes(file_path)
    return pm4py.convert_to_dataframe(log)


<h1> Loading event-logs and transforming</h1>

<h4> Load 2013 incidents dataset </h4>

In [23]:
# Loading dataset, keeping only relevant data, grouping on resource
event_log_2013 = import_xes("datasets/BPI_Challenge_2013_incidents.xes")
df_2013 = event_log_2013[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
df_2013 = df_2013.sort_values(by=['org:resource', 'time:timestamp'])

parsing log, completed traces :: 100%|██████████| 7554/7554 [00:01<00:00, 3926.94it/s]


In [24]:
print(df_2013['org:resource'].nunique())
print(event_log_2013['lifecycle:transition'].nunique())
print(event_log_2013['lifecycle:transition'].unique())


1440
13
['In Progress' 'Awaiting Assignment' 'Resolved' 'Assigned' 'Closed'
 'Wait - User' 'Wait - Implementation' 'Wait' 'Wait - Vendor' 'In Call'
 'Wait - Customer' 'Unmatched' 'Cancelled']


<h4> Loading 2017 dataset </h4>

In [25]:
event_log_2017 = import_xes("datasets/BPI_Challenge_2017.xes")
df_2017 = event_log_2017[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
df_2017 = df_2017.sort_values(by=['org:resource', 'time:timestamp'])
df_2017.info()

parsing log, completed traces :: 100%|██████████| 31509/31509 [00:20<00:00, 1509.29it/s]


<class 'pandas.core.frame.DataFrame'>
Index: 1202267 entries, 0 to 1068834
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype              
---  ------             --------------    -----              
 0   case:concept:name  1202267 non-null  object             
 1   concept:name       1202267 non-null  object             
 2   org:resource       1202267 non-null  object             
 3   time:timestamp     1202267 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 45.9+ MB


In [48]:
print(event_log_2017['concept:name'].nunique())
print(event_log_2017['lifecycle:transition'].unique())

26
['complete' 'schedule' 'withdraw' 'start' 'suspend' 'ate_abort' 'resume']


<h4> Loading the BPIC 2018 event-log </h4>

In [27]:
event_log_2018 = import_xes("datasets/BPI_Challenge_2018.xes")
event_log_2018.info()
event_log_2018.head()
df_2018 = event_log_2018[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
df_2018 = df_2018.sort_values(by=['org:resource', 'time:timestamp'])
df_2018.info()

parsing log, completed traces :: 100%|██████████| 43809/43809 [01:36<00:00, 451.67it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514266 entries, 0 to 2514265
Data columns (total 75 columns):
 #   Column                  Dtype              
---  ------                  -----              
 0   success                 bool               
 1   org:resource            object             
 2   docid_uuid              object             
 3   doctype                 object             
 4   subprocess              object             
 5   docid                   object             
 6   activity                object             
 7   note                    object             
 8   eventid                 object             
 9   identity:id             object             
 10  concept:name            object             
 11  lifecycle:transition    object             
 12  time:timestamp          datetime64[ns, UTC]
 13  case:young farmer       bool               
 14  case:selected_random    bool               
 15  case:penalty_AJLP       bool               
 16  

<h4> Loading the BPIC 2019 event-log </h4>

In [28]:
event_log_2019 = import_xes("datasets/BPI_Challenge_2019.xes")
df_2019 = event_log_2019[['case:concept:name', 'concept:name', 'org:resource', 'time:timestamp']]
df_2019 = df_2019.sort_values(by=['org:resource', 'time:timestamp'])
df_2019.info()

parsing log, completed traces :: 100%|██████████| 251734/251734 [00:31<00:00, 7905.74it/s] 


<class 'pandas.core.frame.DataFrame'>
Index: 1595923 entries, 118143 to 1592235
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype              
---  ------             --------------    -----              
 0   case:concept:name  1595923 non-null  object             
 1   concept:name       1595923 non-null  object             
 2   org:resource       1595923 non-null  object             
 3   time:timestamp     1595923 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 60.9+ MB


In [29]:
print(event_log_2019['org:resource'].nunique())

628


<h1>One-Hot Encoding the event-logs</h1>	


<h4> Apply One-Hot encoding function </h4>

In [ ]:
def apply_ohe_encoding(df):
    df['next_activity'] = df.groupby('org:resource')['concept:name'].shift(-1)
    df = df.dropna(subset=['next_activity'])
    
    # OHE
    df_ohe = pd.get_dummies(df, columns=['concept:name'], prefix='activity')
    df_ohe = pd.get_dummies(df_ohe, columns=['org:resource'], prefix='resource')
    
    # Select Features automatically
    feature_cols = [c for c in df_ohe.columns if c.startswith('activity_') or c.startswith('resource_')]
    
    return df_ohe[feature_cols], df_ohe['next_activity']

<h4> OHE the BPIC 2013 event-log </h4>

In [30]:
df_2013['next_activity'] = df_2013.groupby('org:resource')['concept:name'].shift(-1)
df_2013 = df_2013.dropna(subset=['next_activity'])

# Encoding the 'concept:name' column, as these are the activities
column_to_encode = 'concept:name'

# Apply one-hot encoding using pandas get_dummies
df_2013_OHE = pd.get_dummies(df_2013, columns=[column_to_encode], prefix='activity')
df_2013_OHE = pd.get_dummies(df_2013_OHE, columns=['org:resource'], prefix='resource')

print("Columns in final dataframe 2013:")
print(df_2013_OHE.columns.tolist())

Columns in final dataframe 2013:
['case:concept:name', 'time:timestamp', 'next_activity', 'activity_Accepted', 'activity_Completed', 'activity_Queued', 'activity_Unmatched', 'resource_-', 'resource_Aaron', 'resource_Abby', 'resource_Abdul', 'resource_Abhijit', 'resource_Abhimanyu', 'resource_Abhinav', 'resource_Abhiseka', 'resource_Abhishek', 'resource_Achuthanandan', 'resource_Adam', 'resource_Adeline', 'resource_Adenilson', 'resource_Aditi', 'resource_Adriano', 'resource_Adrien', 'resource_Agata', 'resource_Agneta', 'resource_Agnieszka', 'resource_Agnivesh', 'resource_Agoritsa', 'resource_Agron', 'resource_Ajay', 'resource_Ajitkumar', 'resource_Akanksha', 'resource_Akiko', 'resource_Akim', 'resource_Akshata', 'resource_Ala', 'resource_Alain', 'resource_Alan', 'resource_Alejandro', 'resource_Aleksander', 'resource_Aleksandra', 'resource_Alex', 'resource_Alexander', 'resource_Alexandra', 'resource_Alexandre', 'resource_Alexey', 'resource_Alexsandra', 'resource_Alf', 'resource_Aline', '

<h4> OHE the BPIC 2017 event-log </h4>

In [31]:
df_2017['next_activity'] = df_2017.groupby('org:resource')['concept:name'].shift(-1)
df_2017 = df_2017.dropna(subset=['next_activity'])

# Encoding the 'concept:name' column, as these are the activities
column_to_encode = 'concept:name'

# OHE BPIC 2017 using pandas' get_dummies
df_2017_OHE = pd.get_dummies(df_2017, columns=[column_to_encode], prefix='activity')
df_2017_OHE = pd.get_dummies(df_2017_OHE, columns=['org:resource'], prefix='resource')

print("Columns in final dataframe 2017:")
print(df_2017_OHE.columns.tolist())

Columns in final dataframe 2017:
['case:concept:name', 'time:timestamp', 'next_activity', 'activity_A_Accepted', 'activity_A_Cancelled', 'activity_A_Complete', 'activity_A_Concept', 'activity_A_Create Application', 'activity_A_Denied', 'activity_A_Incomplete', 'activity_A_Pending', 'activity_A_Submitted', 'activity_A_Validating', 'activity_O_Accepted', 'activity_O_Cancelled', 'activity_O_Create Offer', 'activity_O_Created', 'activity_O_Refused', 'activity_O_Returned', 'activity_O_Sent (mail and online)', 'activity_O_Sent (online only)', 'activity_W_Assess potential fraud', 'activity_W_Call after offers', 'activity_W_Call incomplete files', 'activity_W_Complete application', 'activity_W_Handle leads', 'activity_W_Personal Loan collection', 'activity_W_Shortened completion ', 'activity_W_Validate application', 'resource_User_1', 'resource_User_10', 'resource_User_100', 'resource_User_101', 'resource_User_102', 'resource_User_103', 'resource_User_104', 'resource_User_105', 'resource_User_

<h4> OHE the BPIC 2018 event-log </h4>

In [32]:
df_2018['next_activity'] = df_2018.groupby('org:resource')['concept:name'].shift(-1)
df_2018 = df_2018.dropna(subset=['next_activity'])

# Encoding the 'concept:name' column, as these are the activities
column_to_encode = 'concept:name'

# OHE BPIC 2018 using pandas' get_dummies
df_2018_OHE = pd.get_dummies(df_2018, columns=[column_to_encode], prefix='activity')
df_2018_OHE = pd.get_dummies(df_2018_OHE, columns=['org:resource'], prefix='resource')

print("Columns in final dataframe 2018:")
print(df_2018_OHE.columns.tolist())

Columns in final dataframe 2018:
['case:concept:name', 'time:timestamp', 'next_activity', 'activity_abort external', 'activity_abort payment', 'activity_approve', 'activity_begin admissibility check', 'activity_begin editing', 'activity_begin editing from refused', 'activity_begin payment', 'activity_begin preparations', 'activity_calculate', 'activity_calculate protocol', 'activity_cancel offline', 'activity_change department', 'activity_check', 'activity_check admissibility', 'activity_clear', 'activity_correction GFM17', 'activity_create', 'activity_decide', 'activity_discard', 'activity_finish editing', 'activity_finish payment', 'activity_finish pre-check', 'activity_finish preparations', 'activity_initialize', 'activity_insert document', 'activity_mail income', 'activity_mail valid', 'activity_performed', 'activity_performed offline', 'activity_plan', 'activity_prepare external', 'activity_prepare offline', 'activity_refuse', 'activity_remove document', 'activity_restart editing'

<h4> OHE the BPIC 2019 event-log </h4>

In [33]:
df_2019['next_activity'] = df_2019.groupby('org:resource')['concept:name'].shift(-1)
df_2019 = df_2019.dropna(subset=['next_activity'])

# Encoding the 'concept:name' column, as these are the activities
column_to_encode = 'concept:name'

# OHE BPIC 2017 using pandas' get_dummies
df_2019_OHE = pd.get_dummies(df_2019, columns=[column_to_encode], prefix='activity')
df_2019_OHE = pd.get_dummies(df_2019_OHE, columns=['org:resource'], prefix='resource')

print("Columns in final dataframe 2019:")
print(df_2019_OHE.columns.tolist())

Columns in final dataframe 2019:
['case:concept:name', 'time:timestamp', 'next_activity', 'activity_Block Purchase Order Item', 'activity_Cancel Goods Receipt', 'activity_Cancel Invoice Receipt', 'activity_Cancel Subsequent Invoice', 'activity_Change Approval for Purchase Order', 'activity_Change Currency', 'activity_Change Delivery Indicator', 'activity_Change Final Invoice Indicator', 'activity_Change Price', 'activity_Change Quantity', 'activity_Change Rejection Indicator', 'activity_Change Storage Location', 'activity_Change payment term', 'activity_Clear Invoice', 'activity_Create Purchase Order Item', 'activity_Create Purchase Requisition Item', 'activity_Delete Purchase Order Item', 'activity_Reactivate Purchase Order Item', 'activity_Receive Order Confirmation', 'activity_Record Goods Receipt', 'activity_Record Invoice Receipt', 'activity_Record Service Entry Sheet', 'activity_Record Subsequent Invoice', 'activity_Release Purchase Order', 'activity_Release Purchase Requisition'

<h1> 2-Gram encoding the event-logs </h1>

<h4> Creating bigram features function </h4>

In [78]:
def generate_bigram_features(df, window_size=5):
    """
    Splits resource streams into sliding windows and computes
    traditional 2-gram (Bigram) counts for each window.
    """
    print(f"⚙️ Generating 2-gram features (Window Size: {window_size})...")
    
    feature_dicts = [] # List of dictionaries to store counts
    targets = []       # List of next activities
    
    # Process per resource
    for resource, group in df.groupby('org:resource'):
        activities = group['activity'].values
        
        # Sliding window over the resource's timeline
        for i in range(len(activities) - window_size):
            # The context (past N events)
            window = activities[i : i + window_size]
            # The target (next event)
            next_act = activities[i + window_size]
            
            # --- TRADITIONAL 2-GRAM ENCODING LOGIC ---
            # 1. Create pairs: (A, B), (B, C), (C, D)
            bigrams = [
                f"{window[j]}->{window[j+1]}" 
                for j in range(len(window) - 1)
            ]
            
            # 2. Count frequencies (Bag-of-Bigrams)
            # e.g., {'A->B': 1, 'B->A': 2}
            bigram_counts = dict(Counter(bigrams))
            
            feature_dicts.append(bigram_counts)
            targets.append(next_act)

    return feature_dicts, targets

<h4> 2-Gram encoding the 2013 log </h4>

In [83]:
df_2013_2gram = event_log_2013[[
    'case:concept:name', 
    'concept:name', 
    'org:resource', 
    'time:timestamp', 
]].copy()

df_2013_2gram['activity'] = df_2013_2gram['concept:name']

df_2013_2gram = df_2013_2gram.sort_values(by=['org:resource', 'time:timestamp'])

df_2013_2gram = df_2013_2gram.dropna(subset=['org:resource'])

<h4> 2-Gram encoding the 2017 log </h4>

In [84]:
df_2017_2gram = event_log_2017[[
    'case:concept:name', 
    'concept:name', 
    'org:resource', 
    'time:timestamp', 
]].copy()

df_2017_2gram['activity'] = df_2017_2gram['concept:name']

df_2017_2gram = df_2017_2gram.sort_values(by=['org:resource', 'time:timestamp'])

df_2017_2gram = df_2017_2gram.dropna(subset=['org:resource'])

<h1> Training Random Forest model on OHE event-logs </h1>

<h4> Random Forest Training Pipeline </h4>

In [85]:
def train_evaluate_rf(X, y, random_state=1):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )
    
    rf = RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    y_pred = rf.predict(X_test)
    
    return {
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred, average='weighted')
    }

<h4> Training Random forest on 2013 incidents log </h4>


In [86]:
# BPIC 2013 incidents
feature_columns_2013 = [col for col in df_2013_OHE.columns if col.startswith('activity_') or col.startswith('resource')]
X_2013 = df_2013_OHE[feature_columns_2013]
y_2013 = df_2013_OHE['next_activity']

accuracy_2013_OHE, f1_2013_OHE = train_evaluate_rf(X_2013, y_2013)


<h4> Training Random Forest on 2017 log </h4>

In [88]:
# BPIC 2017
feature_columns_2017 = [col for col in df_2017_OHE.columns if col.startswith('activity_') or col.startswith('resource')]
X_2017 = df_2017_OHE[feature_columns_2017]
y_2017 = df_2017_OHE['next_activity']

accuracy_2017_OHE, f1_2017_OHE = train_evaluate_rf(X_2017, y_2017)


<h4>Training Random Forest on 2018 event-log </h4>

In [91]:
# BPIC 2018
feature_columns_2018 = [col for col in df_2018_OHE.columns if col.startswith('activity_') or col.startswith('resource')]
X_2018 = df_2018_OHE[feature_columns_2018]
y_2018 = df_2018_OHE['next_activity']

accuracy_2018_OHE, f1_2018_OHE = train_evaluate_rf(X_2018, y_2018)

<h4> Training Random Forest on 2019 event-log </h4>

In [92]:
# BPIC 2019
feature_columns_2019 = [col for col in df_2019_OHE.columns if col.startswith('activity_') or col.startswith('resource')]
X_2019 = df_2019_OHE[feature_columns_2019]
y_2019 = df_2019_OHE['next_activity']

accuracy_2019_OHE, f1_2019_OHE = train_evaluate_rf(X_2019, y_2019)

<h1> Random Forest Classifier Results </h1>

<h4> One-Hot Encoding results </h4>

In [90]:
# Results OHE 2013 log
print(f"Accuracy 2013: {accuracy_2013_OHE}")
print(f"F1-score 2013: {f1_2013_OHE}")

# Results OHE 2017 log
print(f"Accuracy 2017: {accuracy_2017_OHE}")
print(f"F1-score 2017: {f1_2017_OHE}")


# Results OHE 2018 log
print(f"Accuracy 2018: {accuracy_2018_OHE}")
print(f"F1-score 2018: {f1_2018_OHE}")

# Results OHE 2019 log
print(f"Accuracy 2019: {accuracy_2019_OHE}")
print(f"F1-score 2019: {f1_2019_OHE}")


# Aggregated results 
aggregated_accuracy = statistics.mean([accuracy_2013_OHE,accuracy_2017_OHE, accuracy_2018_OHE, accuracy_2019_OHE])
print(f"Average accuracy is: {aggregated_accuracy}")
aggregated_f1score = statistics.mean([f1score_2013_OHE,f1score_2017_OHE,f1score_2018_OHE,f1score_2019_OHE])
print(f"Average f1-score is: {aggregated_f1score}")

Accuracy 2013: accuracy
F1-score 2013: f1
Accuracy 2017: accuracy
F1-score 2017: f1


NameError: name 'accuracy_2018_OHE' is not defined

<h1> Training Random Forest on 2-gram encoding </h1>

<h4> Training Random Forest on 2-gram encoding on 2013 incidents log </h4>

In [82]:
# D. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_2gram_2017, y_2gram_2017, test_size=0.2, random_state=1)

# E. Random Forest Classifier
print("🚀 Training Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# F. Evaluation
print("📊 Evaluating...")
acc = rf.score(X_test, y_test)
print(f"Accuracy: {acc:.4f}")


🚀 Training Random Forest...
📊 Evaluating...
Accuracy: 0.5622


<h4> Training Random Forest on 2-gram encoding on 2017 log </h4>

In [ ]:
# D. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_2gram_2013, y_2gram_2013, test_size=0.2, random_state=1)

# E. Random Forest Classifier
print("🚀 Training Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# F. Evaluation
print("📊 Evaluating...")
acc = rf.score(X_test, y_test)
print(f"Accuracy: {acc:.4f}")
